**Learning outcomes**

* Describe how the oracle can be applied to a pair of candidate solutions to determine if the secret combination is present.
* Determine the average number of queries required to find a solution when testing in pairs.

In [1]:
import numpy as np
import pennylane as qml

We implement the following computation:
![circuit](./images/A.3.1.png)
The last qubit will in state $\Ket{1}$ if the solution is present, and $\Ket{0}$ if it is not.

**Codercise A.3.1**
 Implement this circuit and return the probabilities on the last qubit. The function `oracle_matrix` is defined for you.

In [2]:
def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2]*len(combo)) # Index of solution
    my_array = np.identity(2**len(combo)) # Create the identity matrix
    my_array[index, index] = -1

    return my_array

In [14]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """Test a pair labelled by x_tilde for the presence of a solution.

    Args:
        x_tilde (list[int]): An (n_bits - 1)-string labelling the pair to test.
        combo (list[int]): A secret combination of n_bits 0s and 1s.

    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)

    oracle = oracle_matrix(combo)

    qml.Hadamard(n_bits-1)
    qml.QubitUnitary(oracle, wires=[i for i in range(n_bits)])
    qml.Hadamard(n_bits-1)

    return qml.probs(wires=n_bits-1)


[1. 0.]


With this circuit at our disposal, we can crack the lock by simpy iterating over the $\tilde{x}$ until we detect the solution. Let's see how long thi takes on average.

**Codercise A.3.2**
COmplete the function below to see how many attempts it takes to break the lock using our quantum circuit. You should find an improvement over the brute force approach, which takes around 9 guesses (on average) for 4 qubits. Note that `pair_circuit` is available.
*Tip* Use `np.isclose(a,b)` to test the probabilities coming from `pair_circuit(x_tilde, combo)`

In [ ]:
def pair_lock_picker(trials):
    """Create a combo, run pair_circuit until it succeeds, and tally success rate.

    Args:
        trials (int): Number of times to test the lock picker.

    Returns:
        float: The average number of times the lock picker uses pair_circuit.
    """
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits) # Random list of bits
        counter = 0
        for x_tilde in x_tildes:
            counter += 1
        prob = pair_circuit(x_tilde, combo)
        if np.isclose(1, prob[1]):
            break

        test_numbers.append(counter)
    return sum(test_numbers)/trials

trials = 500
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")
